# **Import Libraries**

In [15]:
import os
import numpy as np
import cv2
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from scipy.linalg import norm
import matplotlib.pyplot as plt

# **Specify data path**

In [20]:
generated_folder = "Generated_Images"
target_folder = "CUHK/Testing photo"
input_sketches_folder = "CUHK/Testing sketch"

# **Compute Scores**

In [21]:
# Function to compute PSNR
def compute_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

# Function to compute Mean Squared Error (MSE)
def compute_mse(img1, img2):
    return np.mean((img1 - img2) ** 2)

# Function to compute L2 Norm (Euclidean Distance)
def compute_l2_norm(img1, img2):
    return norm(img1.flatten() - img2.flatten())

# Function to compute SSIM (with smaller win_size)
def compute_ssim(img1, img2):
    return ssim(img1, img2, multichannel=True, win_size=3)  # Set smaller window size

In [22]:
# Get list of files in the generated and target image folders and sort them
generated_files = sorted([f for f in os.listdir(generated_folder) if f.endswith(".jpg") or f.endswith(".png")])
target_files = sorted([f for f in os.listdir(target_folder) if f.endswith(".jpg") or f.endswith(".png")])
input_sketch_files = sorted([f for f in os.listdir(input_sketches_folder) if f.endswith(".jpg") or f.endswith(".png")])

In [23]:
# Ensure both folders have the same number of images
if len(generated_files) != len(target_files):
    print(f"Warning: The number of images in the generated folder ({len(generated_files)}) does not match the target folder ({len(target_files)}).")
else:
    # Initialize variables to accumulate total scores
    total_ssim = 0
    total_psnr = 0
    total_mse = 0
    total_l2_norm = 0
    num_images = 0
    # Iterate over generated images and their corresponding target images
    for gen_filename, target_filename, sketch_filename in zip(generated_files, target_files, input_sketch_files):
        gen_path = os.path.join(generated_folder, gen_filename)
        target_path = os.path.join(target_folder, target_filename)
        sketch_path = os.path.join(input_sketches_folder, sketch_filename)

        # Load and preprocess images
        gen_img = img_to_array(load_img(gen_path, target_size=(256, 256)))
        target_img = img_to_array(load_img(target_path, target_size=(256, 256)))
        sketch_img = img_to_array(load_img(sketch_path, target_size=(256, 256)))

        # Convert images to uint8 format for SSIM
        gen_img_uint8 = gen_img.astype(np.uint8)
        target_img_uint8 = target_img.astype(np.uint8)
        sketch_img_uint8 = sketch_img.astype(np.uint8)

        # Compute SSIM (Structural Similarity Index)
        ssim_value = compute_ssim(gen_img_uint8, target_img_uint8)

        # Compute PSNR
        psnr_value = compute_psnr(gen_img_uint8, target_img_uint8)

        # Compute MSE
        mse_value = compute_mse(gen_img_uint8, target_img_uint8)

        # Compute L2 Norm
        l2_norm_value = compute_l2_norm(gen_img_uint8, target_img_uint8)

        # Print the results for each pair
        #print(f"Image Pair: {gen_filename} (Generated) vs {target_filename} (Target) vs {sketch_filename} (Input Sketch)")
        
        '''
        # Plot the images side by side
        plt.figure(figsize=(18, 6))
        
        plt.subplot(1, 3, 1)
        plt.imshow(sketch_img_uint8)
        plt.title(f"Input Sketch")
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(gen_img_uint8)
        plt.title(f"Generated Image\nSSIM: {ssim_value:.4f}")
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(target_img_uint8)
        plt.title(f"Target Image")
        plt.axis('off')
        
        plt.show()
        
        print(f"SSIM: {ssim_value:.4f}, PSNR: {psnr_value:.2f}, MSE: {mse_value:.2f}, L2 Norm: {l2_norm_value:.2f}")
        print("-" * 150)
        
        '''

        # Accumulate scores
        total_ssim += ssim_value
        total_psnr += psnr_value
        total_mse += mse_value
        total_l2_norm += l2_norm_value
        num_images += 1
    # Compute averages
average_ssim = total_ssim / num_images if num_images > 0 else 0
average_psnr = total_psnr / num_images if num_images > 0 else 0
average_mse = total_mse / num_images if num_images > 0 else 0
average_l2_norm = total_l2_norm / num_images if num_images > 0 else 0

# **Result**

In [24]:
print(f"SSIM : {average_ssim}")
print(f"PSNR : {average_psnr}")
print(f"MSE : {average_mse}")
print(f"L2 Norm : {average_l2_norm}")

SSIM : 0.7584196018969425
PSNR : 28.373899324815348
MSE : 94.89643488566082
L2 Norm : 76184.88759803987
